In [1]:
pip install -U pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow-gpu==2.8.0

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install protobuf==3.20

Note: you may need to restart the kernel to use updated packages.


# 데이터셋 로드 및 구조 확인

In [6]:
import pandas as pd
df = pd.read_csv('train data_self_231127.csv')

In [7]:
print('샘플의 개수 :', len(df))

샘플의 개수 : 5100


In [8]:
df.head()

,labels,name,theme,동영상 제목,댓글,댓글 작성일,PRO
0,0,검은사막,게임성,"릴카 피파 하축단 마지막+검은사막 아토락시온 레이드(feat.서새봄,노돌리,소풍왔니...",오늘 영상도 재밌게 시청하겠습니다:),2023-07-30T23:56:32Z,오늘 영상도 재밌게 시청하겠습니다
1,0,검은사막,게임성,"릴카 피파 하축단 마지막+검은사막 아토락시온 레이드(feat.서새봄,노돌리,소풍왔니...",오 아침 업로드 귀하다,2023-07-30T22:10:42Z,오 아침 업로드 귀하다
2,0,리니지,BM,[리니지M] 갑자기 생각도 못한 이벤트를 하네? ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 뭐야 이거 (9...,4개 계정으로 모두 추천했어요 에디션 당첨되시길 ^^,2023-09-27T14:15:45Z,4개 계정으로 모두 추천했어요 에디션 당첨되시길
3,0,제2의나라,BM,"[제2의 나라] 히사이시 조, 제2의 나라를 지휘하다_30s",BHXTHB 추천부탁드립니다 10연뽑기 가져다 드리겠습니다,2021-05-28T11:46:22Z,BHXTHB 추천부탁드립니다 10연뽑기 가져다 드리겠습니다
4,0,제2의나라,BM,[제2의 나라] 코엑스에 거대 고양이 '우다닥'이 나타났다?!,BHXTHB 추천부탁드립니다 10연뽑기 가져다 드리겠습니다ㅎㅎ,2021-05-28T17:23:05Z,BHXTHB 추천부탁드립니다 10연뽑기 가져다 드리겠습니다ㅎㅎ


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  5100 non-null   int64 
 1   name    5100 non-null   object
 2   theme   5100 non-null   object
 3   동영상 제목  5100 non-null   object
 4   댓글      5100 non-null   object
 5   댓글 작성일  5100 non-null   object
 6   PRO     5100 non-null   object
dtypes: int64(1), object(6)
memory usage: 279.0+ KB


In [10]:
from datasets import load_dataset

In [11]:
all_data = load_dataset(
        "csv",
        data_files={
            "train": "train data_self_231127.csv",
        },
    )

In [12]:
all_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'name', 'theme', '동영상 제목', '댓글', '댓글 작성일', 'PRO'],
        num_rows: 5100
    })
})

In [13]:
cs = all_data['train'].train_test_split(0.2)
train_cs = cs["train"]
test_cs = cs["test"]

In [14]:
train_cs

Dataset({
    features: ['labels', 'name', 'theme', '동영상 제목', '댓글', '댓글 작성일', 'PRO'],
    num_rows: 4080
})

In [15]:
test_cs

Dataset({
    features: ['labels', 'name', 'theme', '동영상 제목', '댓글', '댓글 작성일', 'PRO'],
    num_rows: 1020
})

In [16]:
# 훈련 데이터를 다시 8:2로 분리 후 훈련 데이터와 검증 데이터로 저장
cs = train_cs.train_test_split(0.2)
train_cs = cs["train"]
valid_cs = cs["test"]

In [17]:
# 훈련 데이터
train_cs

Dataset({
    features: ['labels', 'name', 'theme', '동영상 제목', '댓글', '댓글 작성일', 'PRO'],
    num_rows: 3264
})

In [18]:
# 검증 데이터
valid_cs

Dataset({
    features: ['labels', 'name', 'theme', '동영상 제목', '댓글', '댓글 작성일', 'PRO'],
    num_rows: 816
})

In [19]:
# 테스트 데이터
test_cs

Dataset({
    features: ['labels', 'name', 'theme', '동영상 제목', '댓글', '댓글 작성일', 'PRO'],
    num_rows: 1020
})

In [20]:
print('두번째 샘플 출력 :', train_cs['PRO'][1])
print('두번째 샘플의 레이블 출력 :', train_cs['labels'][1])

두번째 샘플 출력 : 아  내뚝배기가 저래서 없어지는구나 ㅎ
두번째 샘플의 레이블 출력 : 2


# 데이터셋 전처리

In [21]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import tensorflow as tf
import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [22]:
# 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만듭니다.

train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train_cs['PRO']))
validation_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', valid_cs['PRO']))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test_cs['PRO']))

In [23]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']

In [24]:
test_sentences[:5]

['[CLS] 랑 이랑 데미지  배 차이 나는구나 [SEP]',
 '[CLS] 예시 사냥터가 할일이 전혀없는곳이라 도움이 안되네요    [SEP]',
 '[CLS] 아니    너무 잘그리잖아요    [SEP]',
 '[CLS] 원래 트레일러는 잘 안보는데 마동석때문에 본다ㅋㅋㅋ [SEP]',
 '[CLS] 내가볼때 그냥 병신같은게임문화다    [SEP]']

In [25]:
test_labels[:5]

[0, 2, 2, 2, 2]

# Electra 토크나이저를 이용한 전처리

In [26]:
from transformers import ElectraTokenizer
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [27]:
MAX_LEN = 128

def data_to_tensor (sentences, labels):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 0으로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 0, 0, 0]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks

In [28]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [29]:
print(train_inputs[0])
print(train_masks[0])

tensor([    2,  2783,  4256,  4073, 19339,  7389, 21588, 13335,  4279,  7599,
        25638,  4039,  6863, 32603, 11318,  6571,  2411,  4065,  4150,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [30]:
tokenizer.decode([2])

'[CLS]'

In [31]:
tokenizer.decode([3])

'[SEP]'

In [32]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [33]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 3264
검증 데이터의 크기: 816
테스트 데이터의 크기: 1020


# GPU가 정상 셋팅되었는지 확인.

In [34]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [35]:
device

device(type='cuda')

# 모델 로드하기

In [36]:
num_labels = 3

In [37]:
from transformers import ElectraForSequenceClassification

model = ElectraForSequenceClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator", num_labels=num_labels)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [38]:
model.cuda()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

# 모델 학습

In [39]:
epochs = 50
batch_size = 16

In [40]:
from torch.nn import functional as F

In [41]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

/home/ubuntu/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [42]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device),
                   attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

 49%|████▉     | 100/204 [00:30<00:30,  3.42it/s]

Batch Loss: 109.22382283210754 Accuracy: tensor(0.3738, device='cuda:0')


 98%|█████████▊| 200/204 [00:59<00:01,  3.38it/s]

Batch Loss: 212.7547683119774 Accuracy: tensor(0.4456, device='cuda:0')


100%|██████████| 204/204 [01:00<00:00,  3.36it/s]


...

Batch Loss: 0.955335469392594 Accuracy: tensor(0.9969, device='cuda:0')


 98%|█████████▊| 200/204 [00:59<00:01,  3.34it/s]

Batch Loss: 1.4144206609053072 Accuracy: tensor(0.9981, device='cuda:0')


100%|██████████| 204/204 [01:01<00:00,  3.34it/s]

Train Loss: 1.4296517690818291 Accuracy: tensor(0.9982, device='cuda:0')


In [43]:
losses, accuracies

([216.8673450946808,
  183.8015902042389,
  149.20192208886147,
  123.84090501070023,
  100.49428327381611,
  81.7977334856987,
  64.6983347311616,
  50.37941005080938,
  38.40815966948867,
  30.65870031528175,
  23.94121809490025,
  21.464309750124812,
  18.266466938890517,
  14.451540576294065,
  12.251824893988669,
  10.896443322300911,
  10.892626154236495,
  8.67393829766661,
  7.3668147451244295,
  7.561801929725334,
  6.793330653570592,
  5.421665989793837,
  4.881592125399038,
  4.985962258186191,
  4.324281042558141,
  4.920414556283504,
  3.3255133860511705,
  3.4974925734568387,
  2.3832142328610644,
  2.351376660633832,
  1.6945557881845161,
  2.6316580157144926,
  3.4003812354640104,
  3.5964867185684852,
  2.878088054188993,
  2.286498607834801,
  2.2881355810095556,
  2.1308115960564464,
  2.4844883881451096,
  1.71885228579049,
  0.7261196915933397,
  2.5580122816900257,
  2.4514674765523523,
  1.1224084021814633,
  3.9770521726750303,
  3.8207419220125303,
  0.78212116

# 테스트 데이터에 대한 평가

In [44]:
model.eval()
accum_logits, accum_label_ids = [], []
test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device),
                   attention_mask=attention_masks_batch.to(device)
                  )[0]
    _, predicted = torch.max(y_pred, 1)
    for b in y_pred.cpu().detach().numpy():
        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정
        # ex) [ 3.5134246  -0.30875662 -2.111316  ] ==> 0
        accum_logits.append(np.argmax(b))

    for b in y_batch.cpu().detach().numpy():
        accum_label_ids.append(b)
    
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

100%|██████████| 64/64 [00:06<00:00,  9.73it/s]

Accuracy: tensor(0.7824, device='cuda:0')


# 메트릭 출력

In [45]:
def metrics(predictions, labels):
    y_pred = predictions
    y_true = labels

    # 사용 가능한 메트릭들을 사용한다.
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # 메트릭 결과에 대해서 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average}

    return metrics

In [46]:
accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

In [47]:
print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))

Accuracy: 0.7824
F1 (Macro) Score: 0.7805
F1 (Micro) Score: 0.7824
F1 (Weighted) Score: 0.7803


In [48]:
# 모델 저장하기
torch.save(model.state_dict(), "model_self.pt")

# 예측

In [49]:
import numpy as np
from tqdm import tqdm

In [50]:
df = pd.read_csv('./4th_Add Data Colums.csv')

In [51]:
df.head()

,labels,name,theme,동영상 제목,댓글,댓글 작성일,PRO
0,,메이플,BM,"환산10만 루미 장비 및 보스 레이드 [메이플스토리, 뚝이]",환산10만할라면 현질얼마해야되나요,2023-10-07T06:58:56Z,환산10만할라면 현질얼마해야되나요
1,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],11:00 채팅창 보니까 쉘로 넘겼다는 말 하나도 없네;;;,2023-06-20T13:11:19Z,채팅창 보니까 쉘로 넘겼다는 말 하나도 없네
2,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,2023-03-18T01:42:56Z,영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ
3,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],그리고 2년지나고 이재야 생겨난 큐브 천장,2023-04-14T10:01:43Z,그리고 2년지나고 이재야 생겨난 큐브 천장
4,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,2023-03-22T19:55:52Z,근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61220 entries, 0 to 61219
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  50161 non-null  object
 1   name    61220 non-null  object
 2   theme   61220 non-null  object
 3   동영상 제목  61220 non-null  object
 4   댓글      61220 non-null  object
 5   댓글 작성일  61220 non-null  object
 6   PRO     61220 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


In [53]:
from transformers import pipeline

In [54]:
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512,
                return_all_scores=True, function_to_apply='softmax')

/home/ubuntu/anaconda3/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [55]:
label_dict = {'LABEL_0' : '중립', 'LABEL_1' : '긍정', 'LABEL_2' : '부정'}

In [56]:
# def prediction(text):
#   result = pipe(text)

#   return [label_dict[result[0]['label']]]

In [57]:
def prediction(text):
    results = pipe(text)
    # 가장 높은 점수를 가진 레이블을 찾습니다.
    highest_score_label = max(results[0], key=lambda x: x['score'])['label']
    return label_dict[highest_score_label]

In [58]:
# df["PRO"] 열을 500개씩 나누어 처리
chunks = np.array_split(df['PRO'], np.ceil(len(df) / 500))

# 각 청크에 대해 예측 수행 및 진행 상황을 tqdm으로 표시
predicted_labels = []
for chunk in tqdm(chunks, desc="Predicting"):
    predicted_labels.extend(chunk.apply(prediction))

# 예측된 레이블을 새로운 열에 저장
df['예측 결과_self'] = predicted_labels

Predicting:   0%|          | 0/123 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ubuntu/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Predicting: 100%|██████████| 123/123 [09:15<00:00,  4.52s/it]


In [59]:
df['예측 결과_self'].value_counts()

중립    24271
부정    20626
긍정    16323
Name: 예측 결과_self, dtype: int64

In [60]:
df

,labels,name,theme,동영상 제목,댓글,댓글 작성일,PRO,예측 결과_self
0,,메이플,BM,"환산10만 루미 장비 및 보스 레이드 [메이플스토리, 뚝이]",환산10만할라면 현질얼마해야되나요,2023-10-07T06:58:56Z,환산10만할라면 현질얼마해야되나요,중립
1,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],11:00 채팅창 보니까 쉘로 넘겼다는 말 하나도 없네;;;,2023-06-20T13:11:19Z,채팅창 보니까 쉘로 넘겼다는 말 하나도 없네,부정
2,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,2023-03-18T01:42:56Z,영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,긍정
3,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],그리고 2년지나고 이재야 생겨난 큐브 천장,2023-04-14T10:01:43Z,그리고 2년지나고 이재야 생겨난 큐브 천장,중립
4,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,2023-03-22T19:55:52Z,근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,중립
...,...,...,...,...,...,...,...,...
61215,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",잼있겠다.. ㅠㅠ 나도 하고싶다,2023-06-30T03:48:27Z,잼있겠다 나도 하고싶다,중립
61216,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",저녀석 잡는거 다시보여주세용ㅋㅋ 오랫만에보니재밋넹,2023-06-01T14:26:42Z,저녀석 잡는거 다시보여주세용ㅋㅋ 오랫만에보니재밋넹,긍정
61217,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",저도 요정 하고있는데 스톰샷 마법이 클릭하면 안배워지고 자꾸 땅에 떨어지는데 어떻...,2023-07-08T13:04:39Z,저도 요정 하고있는데 스톰샷 마법이 클릭하면 안배워지고 자꾸 땅에 떨어지는데 어떻...,중립
61218,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",형 가디언 잡는거 보여줘...,2023-07-14T09:14:21Z,형 가디언 잡는거 보여줘,중립


In [61]:
df.to_csv('231201_Self_final.csv', index=False, encoding='utf-8')